In [ ]:
#First import and upload the zip file Shared with the github
!pkill -f streamlit; !pkill -f uvicorn; !pkill -f ngrok
import time; time.sleep(3)

!pip install -q streamlit pyngrok transformers torch pandas plotly wordcloud matplotlib fastapi uvicorn

# ===== API =====
import os
os.makedirs("api", exist_ok=True)
with open("api/main.py", "w") as f:
    f.write('''
from fastapi import FastAPI
from transformers import pipeline

app = FastAPI()

sentiment = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
cats = ["billing","support","product","service","account"]

@app.post("/predict")
def predict(data: dict):
    text = data["text"]
    result = sentiment(text)[0]
    raw_label = result["label"]  # "negative", "neutral", "positive"
    c = zero_shot(text, cats)["labels"][0]
    return {"text": text[:80], "sentiment": raw_label, "category": c}
''')

# ===== STREAMLIT DASHBOARD =====
with open("app.py", "w") as f:
    f.write('''
import streamlit as st, requests, time, pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

st.set_page_config(page_title="LIVE", layout="wide")
st.title("COMPLAINTS FLYING IN — SENTIMENT FIXED ✅")

live = st.empty()
charts = st.empty()
wc = st.empty()
final = st.empty()

neg_words = []
cat_count = {"billing":0, "support":0, "product":0, "service":0, "account":0}
pos = 0
neg = 0
neu = 0

df = pd.read_parquet("complaintpro/data/processed/complaints.parquet").sample(20)

for i, txt in enumerate(df["clean_text"]):
    try:
        r = requests.post("http://localhost:8000/predict", json={"text": txt})
        d = r.json()
        live.success(f"{i+1}/20 → {d['category'].upper()} | {d['sentiment']} | {d['text']}...")

        # ✅ FIXED: sentiment mapping handles lowercase words
        cat_count[d["category"]] += 1
        sent = d["sentiment"].lower()
        if "neg" in sent:
            neg += 1
            neg_words.append(d["text"])
        elif "pos" in sent:
            pos += 1
        else:
            neu += 1

        # Charts
        with charts.container():
            col1, col2 = st.columns(2)
            col1.plotly_chart(
                px.bar(x=list(cat_count.keys()), y=list(cat_count.values()),
                       title="Live Categories", color_discrete_sequence=["#636EFA"]),
                use_container_width=True
            )
            col2.plotly_chart(
                px.pie(values=[pos, neg, neu], names=["POSITIVE","NEGATIVE","NEUTRAL"],
                       title="Live Sentiment", color_discrete_sequence=["#00CC96","#FF4444","#636EFA"]),
                use_container_width=True
            )

        if len(neg_words) > 2:
            cloud = WordCloud(width=400, height=300, background_color="black", colormap="Reds").generate(" ".join(neg_words))
            fig, ax = plt.subplots()
            ax.imshow(cloud); ax.axis("off")
            wc.pyplot(fig)

        time.sleep(1.5)
    except Exception as e:
        print("Error:", e)
        time.sleep(1)

with final.container():
    st.balloons()
    st.success("20 COMPLAINTS ANALYZED!")
    col1, col2, col3 = st.columns(3)
    col1.metric("Total", 20)
    col2.metric("Positive", pos)
    col3.metric("Negative", neg)

    st.subheader("FINAL CATEGORIES")
    st.plotly_chart(px.bar(x=list(cat_count.keys()), y=list(cat_count.values())))

    st.subheader("FINAL SENTIMENT")
    st.plotly_chart(
        px.pie(values=[pos, neg, neu], names=["POSITIVE","NEGATIVE","NEUTRAL"],
               color_discrete_sequence=["#00CC96","#FF4444","#636EFA"])
    )
''')

# ===== START EVERYTHING =====
import subprocess
subprocess.Popen(["uvicorn", "api.main:app", "--host", "0.0.0.0", "--port", "8000"])
dash = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])
time.sleep(10)

from pyngrok import ngrok
ngrok.set_auth_token("34X9HMFVyMSnzcz2Rjw1BfwyHga_4YuVVEGP2zxJWaneqL2Lt")  # Replace with your ngrok token
dash_url = ngrok.connect(8501).public_url
print(f"\\nLIVE DASHBOARD → {dash_url}")
print("✅ Sentiment Chart + Negative Count Updating LIVE Now!")
